block

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

X=torch.rand(2,20)
X

tensor([[0.8339, 0.7377, 0.3304, 0.8273, 0.5998, 0.5202, 0.5145, 0.5623, 0.9538,
         0.0434, 0.0862, 0.2758, 0.9156, 0.3644, 0.8501, 0.0588, 0.3622, 0.5903,
         0.1060, 0.9224],
        [0.2401, 0.7765, 0.7189, 0.8662, 0.2179, 0.1140, 0.2388, 0.5746, 0.1126,
         0.2345, 0.8454, 0.4761, 0.0684, 0.0511, 0.5048, 0.2414, 0.2112, 0.2316,
         0.1075, 0.4251]])

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
        
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.0270, -0.1526,  0.1548,  0.0859, -0.1763,  0.1204, -0.2043, -0.2455,
         -0.0254, -0.1840],
        [ 0.0676,  0.0232, -0.0430,  0.0758, -0.0276,  0.1455, -0.0246, -0.1385,
         -0.1412, -0.0969]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self,X):
        X=self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.1615, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1432, grad_fn=<SumBackward0>)

参数管理

In [12]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1212],
        [-0.1956]], grad_fn=<AddmmBackward0>)

In [13]:
print(net[2].state_dict())

OrderedDict({'weight': tensor([[ 0.2534,  0.1753, -0.1055, -0.3068, -0.1566, -0.1630,  0.0735, -0.0590]]), 'bias': tensor([-0.0800])})


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0800], requires_grad=True)
tensor([-0.0800])


In [15]:
net[2].weight.grad == None

True

In [17]:
print(*[(name,param.shape) for name ,param in net[0].named_parameters()])
print(*[(name,param.shape) for name ,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:
net.state_dict()['2.bias'].data

tensor([-0.0800])

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)


tensor([[-0.3126],
        [-0.3126]], grad_fn=<AddmmBackward0>)

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [21]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0104,  0.0040, -0.0236, -0.0089]), tensor(0.))

In [22]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [23]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0097, -0.3153,  0.0753, -0.2774])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [24]:
def my_init(m):
    if type(m)==nn.Linear:
        print('Init',*[(name,param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -0.0000, -7.9206,  8.6056],
        [-9.9488,  6.8639, -5.0910,  6.0684]], grad_fn=<SliceBackward0>)

In [25]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -6.9206,  9.6056])

In [26]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


自定义层

In [27]:
import torch
import torch.nn.functional as F
from torch import nn
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()



In [28]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [29]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [42]:
Y = net(torch.randn(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

In [43]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)


In [44]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.9930,  0.2771,  0.4575],
        [ 1.2737, -0.6129, -1.4377],
        [-1.3857,  0.1024, -2.3637],
        [-0.3400,  0.4485,  0.7384],
        [-0.4616,  1.0556,  1.1626]], requires_grad=True)

In [45]:
linear(torch.rand(2, 5))

tensor([[1.1278, 1.3803, 0.0000],
        [0.7868, 1.2493, 0.0000]])

In [46]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

读写文件

In [47]:
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4)
torch.save(x, 'x-file')

In [48]:
x2=torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [49]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)


(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [50]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [51]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net=MLP()
X=torch.randn(size=(2,20))
Y=net(X)

In [52]:
torch.save(net.state_dict(),'mlp.params')

In [53]:
clone=MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [55]:
Y_clone=clone(X)
Y_clone==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])